# Hadoop Streaming assignment 2: Stop Words

In [24]:
%%writefile mapper3.py

import sys
import re


path = 'stop_words_en.txt'

# Your code for reading stop words here
stop_words = set(word.strip() for word in open(path))

for line in sys.stdin:
    try:
        article_id, text = line.strip().split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        if word.lower() in stop_words:
            sys.stderr.write(f'reporter:counter:Wiki stats,Stop words,1\n')
        sys.stderr.write(f'reporter:counter:Wiki stats,Total words,1\n')
        print(f'{word.lower()}\t1')

Overwriting mapper3.py


In [19]:
%%writefile reducer3.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer3.py


In [27]:
%%writefile counter_process.py

import sys
import re

stop_counter = sys.argv[1]
total_counter = sys.argv[2]

for line in sys.stdin:
    if re.search(stop_counter, line):
        stop_words_cnt = int(re.findall('\d+', line)[0])
    if re.search(total_counter, line):
        total_words_cnt = int(re.findall('\d+', line)[0])

#print("stop_words : ",stop_words_cnt)
print(round(stop_words_cnt/total_words_cnt*100,3))

Overwriting counter_process.py


In [7]:
%%bash

OUT_DIR="result_stopwords"
NUM_REDUCERS=4
LOGS="stderr_logs.txt"
hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Calculating Stop words" \
    -D mapreduce.job.reduces=0 \
    -files mapper3.py,/datasets/stop_words_en.txt \
    -mapper "python3 mapper3.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS

cat $LOGS | python3 ./counter_process.py "Stop words" "Total words"
cat $LOGS >&2

rm: `result_stopwords': No such file or directory
bash: line 6: output.log: Permission denied
Traceback (most recent call last):
  File "counter_process.py", line 18, in <module>
    print(ratioStopTotal(sys.argv[1], sys.argv[2]))
  File "counter_process.py", line 13, in ratioStopTotal
    return round(float(flag_stop*100)/flag_nonstop,3)
UnboundLocalError: local variable 'flag_stop' referenced before assignment
